<a href="https://colab.research.google.com/github/vibhutir/Melanoma-Detection-with-Domain-Specific-Artifact-Debiasing-and-Augmentation/blob/main/Copy_of_Hair_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import albumentations as albu
import tensorflow as tf

In [ ]:
hair_pic = files.upload()

NameError: ignored

In [ ]:
no_hair_pic = files.upload()

In [ ]:
size=1024

hair = cv2.imread('ISIC_0000019_downsampled (1).jpg')
image_resize_hair = cv2.resize(hair,(size,size))
    
image_resize_hair = cv2.cvtColor(image_resize_hair,cv2.COLOR_BGR2RGB)
plt.imshow(image_resize_hair)

In [ ]:
size=1024

no_hair = cv2.imread('ISIC_0000008 (1).jpg')
image_resize_no_hair = cv2.resize(no_hair,(size,size))
    
image_resize_no_hair = cv2.cvtColor(image_resize_no_hair,cv2.COLOR_BGR2RGB)
plt.imshow(image_resize_no_hair)

In [ ]:
#extracting hair mask from hairy image
lower_limit = 20 # the value that I found helpful

#*********#*********PROCEDURE*********#*********#*********#
###################################
grayScale = cv2.cvtColor(image_resize_hair, cv2.COLOR_RGB2GRAY)

# Kernel for the    
kernel = cv2.getStructuringElement(1,(17,17))

# Perform the blackHat filtering on the grayscale image to find the hair countours
blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)

# intensify the hair countours  
_ ,threshold = cv2.threshold(blackhat,20,255,cv2.THRESH_BINARY)
#######################################
threshold = cv2.bitwise_not(threshold)
plt.imshow(threshold,cmap = 'gray')


In [ ]:
plt.title('no hair image with hair mask on it')
plt.imshow(cv2.bitwise_and(image_resize_no_hair,image_resize_no_hair,mask = threshold))

In [ ]:
# Run this cell to mount Google Drive for Colab
from google.colab import drive
drive.mount('/content/drive/')
# !ls '/content/drive/My Drive/Colab Notebooks'

import os
os.chdir('/content/drive/Shareddrives/Colab Notebooks/isic-2019')

In [ ]:
!unzip -u "/content/drive/My Drive/Colab Notebooks/isic-2019/ISIC_2019_Training_Input.zip" -d "/content/drive/My Drive/Colab Notebooks/isic-2019/ISIC_2019_Training_Input"

In [ ]:
BASE_PATH = '/content/drive/My Drive/Colab Notebooks/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input'
#list of images with hair
#hair_images =['ISIC_0078712','ISIC_0080817','ISIC_0082348','ISIC_0109869','ISIC_0155012','ISIC_0159568','ISIC_0164145','ISIC_0194550','ISIC_0194914','ISIC_0202023','ISIC_0083035','ISIC_0068279','ISIC_0109703','ISIC_0149527']
hair_images = ['ISIC_0030107', 'ISIC_0030169','ISIC_0030167','ISIC_0030218','ISIC_0030234','ISIC_0030132','ISIC_0030127','ISIC_0030120','ISIC_0030097','ISIC_0030072','ISIC_0030167','ISIC_0031222','ISIC_0031180','ISIC_0030524']

In [ ]:
size=1024
for img in hair_images:
    image = cv2.imread(BASE_PATH + '/' + img + '.jpg')
    image_resize = cv2.resize(image,(size,size))
    image_resize = cv2.cvtColor(image_resize,cv2.COLOR_BGR2RGB)
    plt.imshow(image_resize)
    plt.show()

error: ignored

In [ ]:
#Extracting hair masks from images
def img(image_name,lower_limit=20):    
    '''
    Helper Function to help us iterate with our code!!   
    
    
    '''

    image = cv2.imread(BASE_PATH + '/' + image_name + '.jpg')
    image_resize = cv2.resize(image,(size,size))
   
    grayScale = cv2.cvtColor(image_resize, cv2.COLOR_RGB2GRAY)
      # Kernel for the morphological filtering
    kernel = cv2.getStructuringElement(1,(17,17))

    # Perform the blackHat filtering on the grayscale image to find the hair countours
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)

    # intensify the hair countours in preparation for the inpainting 
    _ ,threshold = cv2.threshold(blackhat,lower_limit,255,cv2.THRESH_BINARY)
    # inpaint the original image depending on the mask
    final_image = cv2.inpaint(image_resize,threshold,1,cv2.INPAINT_TELEA)
    
    threshold = cv2.bitwise_not(threshold)
    image_resize = cv2.cvtColor(image_resize,cv2.COLOR_BGR2RGB)
    final_image = cv2.cvtColor(final_image,cv2.COLOR_BGR2RGB)
    
    return image_resize,threshold,final_image

In [ ]:
image_2,hair_mask_2,_ = img(hair_images[1])
plt.title('The second image')
plt.imshow(image_2)
plt.show()

In [ ]:
plt.title('The hair mask of the second image')
plt.imshow(hair_mask_2,cmap = 'binary_r')
plt.show()

In [ ]:
image_1,_,_ = img(hair_images[0]) 
plt.title('The second image')
plt.imshow(image_1)
plt.show()

In [ ]:
for i,img_name in enumerate(hair_images) :
    _,hair_mask,_ = img(img_name)
    plt.title(f'{i},{img_name}')
    plt.imshow(cv2.bitwise_and(image_1,image_1,mask = hair_mask))
    plt.show()

In [ ]:
possible_cands = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]# possible candidates 
all_hair_masks = []
for i,img_id in enumerate(possible_cands):
    _,hair_masks,_ = img(hair_images[img_id])
    all_hair_masks.append(hair_masks) 
    cv2.imwrite(f'image_{i}.jpg',hair_masks)
    print(len(all_hair_masks))

In [ ]:
all_hair_masks = np.array(all_hair_masks)
np.save('hair_array.npy',all_hair_masks.astype(np.uint8))


In [ ]:
#loading the hairs
hairs = np.load('hair_array.npy')
#the random transformations we want to apply on the masks
hair_trans = albu.Compose([
    albu.ShiftScaleRotate(rotate_limit=[-45,45],scale_limit=[-0.1,0.1],
                          shift_limit=[-0.1,0.15],border_mode=3,p=1.)])

FileNotFoundError: ignored

In [ ]:
from numpy.random import choice
def hair_mask(hairs,IMAGE_SIZE,p = 0.3):
    chance = np.random.uniform(0,1,1)
    if chance <= p:
        mask_to_chose = choice(np.arange(14), 1,p=[0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.075,0.075,0.075,0.075])[0]
        mask = hairs[mask_to_chose]
        
        mask = hair_trans(image = mask)['image']
        mask = cv2.resize(mask/255,(IMAGE_SIZE,IMAGE_SIZE),cv2.INTER_CUBIC)
        mask[mask == 1.] =  255
        mask[mask != 255.] = 0
    else:
        mask = np.ones((IMAGE_SIZE,IMAGE_SIZE))
    return mask

In [ ]:
msk = hair_mask(hairs,IMAGE_SIZE=256,p=1.).astype(np.uint8)
plt.imshow(msk,cmap = 'binary_r')

In [ ]:
img = no_hair
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img,(256,256))

plt.imshow(cv2.bitwise_and(img,img,mask= msk))

In [ ]:
#Use this in data loader!!
#composition = albu.Compose([albu.ShiftScaleRotate(rotate_limit=[-90,90],scale_limit=[-0.42,0.35],shift_limit=0,border_mode=0,p=0.5),])
#image = composition(image=img)['image']
#msk = hair_mask(hairs,IMAGE_SIZE=256,p=1.).astype(np.uint8)
#image = cv2.bitwise_and(img,img,mask= msk)